In [9]:
import numpy as np
import pandas as pd
import random


In [10]:
VM_catalogue = pd.read_excel("VM_Catalogue.xlsx",skiprows=1)
workload = pd.read_excel("workload.xlsx")
CPU_VM = VM_catalogue["CPU"].values
RAM_VM = VM_catalogue["RAM"].values
COUT_VM = VM_catalogue["Cost"].values
DISPO_VM = VM_catalogue["Quantity"].values
print(CPU_VM)
# Extraction des paramètres composants
ReqCPU = np.array([1, 1, 1, 1])
ReqRAM = np.array([0.25, 0.25, 0.25, 0.25])
mu = np.array([1416, 696, 1005, 1259])

# λ_i = max de la trace
lamb = workload.max()["input_rate"]
print(lamb)

[  1   2   2   2   4   4   4   8   8   8  16  16  16  40  80  96 160]
5751


In [11]:
NB_COMP = len(ReqCPU)
NB_VM = len(CPU_VM)
print(NB_VM)
def replicas_minimales():
    """Calcule r_i minimal pour la stabilité."""
    return np.ceil(lamb / mu).astype(int)

17


In [28]:
def lambda_scaled(k):
    return k * lamb


In [29]:
def replicas_minimales_k(k):
    return np.ceil((k * lamb) / mu).astype(int)


In [12]:
def placement_glouton(r, x):
    """
    Construit y_ij si possible.
    Retourne y ou None si impossible.
    """
    y = np.zeros((NB_COMP, NB_VM), dtype=int)
    ordre_vm = np.argsort(COUT_VM)  # VM les moins chères d'abord

    for i in range(NB_COMP):
        reste = r[i]
        for j in ordre_vm:
            cpu_util = np.dot(y[:, j], ReqCPU)
            ram_util = np.dot(y[:, j], ReqRAM)

            max_cpu = (x[j]*CPU_VM[j] - cpu_util) // ReqCPU[i]
            max_ram = (x[j]*RAM_VM[j] - ram_util) // ReqRAM[i]
            max_possible = int(min(max_cpu, max_ram, reste))

            if max_possible > 0:
                y[i, j] += max_possible
                reste -= max_possible

            if reste == 0:
                break

        if reste > 0:
            return None

    return y


In [13]:
def cout_total(x):
    return np.sum(x * COUT_VM)

def faisable(r, x):
    return placement_glouton(r, x) is not None


In [52]:
def reparer(r, x):
    """Répare une solution en garantissant la faisabilité."""
    
    # Stabilité
    r = np.maximum(r, replicas_minimales())

    # Disponibilité
    x = np.minimum(x, DISPO_VM)

    # Ajouter des VMs tant que placement impossible
    essais = 0
    while placement_glouton(r, x) is None and essais < 100:
        # Choisir la VM la plus rentable
        j_best = np.argmin(COUT_VM / CPU_VM)

        if x[j_best] < DISPO_VM[j_best]:
            x[j_best] += 1
        else:
            # Essayer une autre VM
            j_alt = random.randint(0, NB_VM-1)
            if x[j_alt] < DISPO_VM[j_alt]:
                x[j_alt] += 1
        essais += 1

    if placement_glouton(r, x) is None:
        return None, None

    return r, x


In [31]:
def solution_initiale():
    """Construit une solution initiale toujours faisable."""
    
    # Réplicas minimaux (après scaling)
    r = replicas_minimales().copy()

    # Essai simple : choisir une VM la plus rentable
    j_best = np.argmin(COUT_VM / CPU_VM)
    x = np.zeros(NB_VM, dtype=int)

    # On provisionne assez de CPU pour couvrir la demande
    cpu_total = np.sum(r * ReqCPU)
    x[j_best] = int(np.ceil(cpu_total / CPU_VM[j_best]))

    # Tenter une réparation
    r2, x2 = reparer(r, x)

    # Si échec → stratégie alternative
    if r2 is None:
        # On essaye une initialisation aléatoire (mais raisonnable)
        x_random = np.zeros(NB_VM, dtype=int)
        for j in range(NB_VM):
            x_random[j] = random.randint(0, 3)

        r2, x2 = reparer(r, x_random)

        # Si encore None → boucle jusqu'à en trouver une valide
        essais = 0
        while r2 is None and essais < 50:
            x_random = np.random.randint(0, 3, size=NB_VM)
            r2, x2 = reparer(r, x_random)
            essais += 1

        if r2 is None:
            raise RuntimeError("Impossible de construire une solution initiale faisable.")

    return r2, x2


In [16]:
def voisin(r, x):
    r2, x2 = r.copy(), x.copy()

    if random.random() < 0.5:
        # Modifier un réplica
        i = random.randint(0, NB_COMP-1)
        r2[i] = max(1, r2[i] + random.choice([-1, 1]))
    else:
        # Modifier un nombre de VM
        j = random.randint(0, NB_VM-1)
        x2[j] = max(0, x2[j] + random.choice([-1, 1]))

    return r2, x2


In [17]:
def hill_climbing(nb_iter=2000):

    r, x = solution_initiale()
    meilleur_cout = cout_total(x)

    for _ in range(nb_iter):
        r2, x2 = voisin(r, x)
        r2, x2 = reparer(r2, x2)

        if r2 is None:
            continue

        nouveau_cout = cout_total(x2)

        if nouveau_cout < meilleur_cout:
            r, x = r2, x2
            meilleur_cout = nouveau_cout

    y = placement_glouton(r, x)
    return r, x, y, meilleur_cout


In [18]:
r_opt, x_opt, y_opt, c_opt = hill_climbing()

print("Coût optimal :", c_opt)
print("Répliques optimales :", r_opt)
print("VM optimales :", x_opt)
print("Placement y_ij :\n", y_opt)


Coût optimal : 1.7648
Répliques optimales : [5 9 6 5]
VM optimales : [0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0]
Placement y_ij :
 [[0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 9 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0]]


ALGO GA

In [19]:
def individu_aleatoire():
    """Génère une solution (r, x) aléatoire mais raisonnable."""
    # Réplicas initiaux au minimum requis
    r = replicas_minimales().copy()

    # On ajoute parfois +1 réplica aléatoirement
    for i in range(NB_COMP):
        if random.random() < 0.3:
            r[i] += random.randint(0, 2)

    # Choix aléatoire des VMs
    x = np.zeros(NB_VM, dtype=int)
    for j in range(NB_VM):
        if random.random() < 0.3:
            x[j] = random.randint(0, 3)

    # Réparer pour garantir faisabilité
    r, x = reparer(r, x)
    if r is None:
        return individu_aleatoire()
    
    return {"r": r, "x": x, "cout": cout_total(x)}


In [20]:
def selection(population, k=3):
    """Sélection par tournoi."""
    candidats = random.sample(population, k)
    return min(candidats, key=lambda ind: ind["cout"])


In [21]:
def crossover(parent1, parent2):
    """Croisement simple entre deux parents."""
    r1, x1 = parent1["r"], parent1["x"]
    r2, x2 = parent2["r"], parent2["x"]

    # Croisement mono-point
    point_r = random.randint(1, NB_COMP-1)
    point_x = random.randint(1, NB_VM-1)

    r_child = np.concatenate([r1[:point_r], r2[point_r:]])
    x_child = np.concatenate([x1[:point_x], x2[point_x:]])

    return r_child, x_child


In [22]:
def mutation(r, x, taux=0.2):
    """Mutation légère sur r et x."""
    r2, x2 = r.copy(), x.copy()

    # Mutation réplicas
    for i in range(NB_COMP):
        if random.random() < taux:
            r2[i] = max(1, r2[i] + random.choice([-1, 1]))

    # Mutation VMs
    for j in range(NB_VM):
        if random.random() < taux:
            x2[j] = max(0, x2[j] + random.choice([-1, 1]))

    return r2, x2


In [23]:
def enfant(parent1, parent2):
    r, x = crossover(parent1, parent2)
    r, x = mutation(r, x)

    # Réparer les contraintes
    r, x = reparer(r, x)
    if r is None:
        return enfant(parent1, parent2)

    return {"r": r, "x": x, "cout": cout_total(x)}


In [37]:
def genetic_algorithm(taille_pop= 80, generations=200):
    # Initialisation population
    population = [individu_aleatoire() for _ in range(taille_pop)]

    meilleur = min(population, key=lambda ind: ind["cout"])

    for gen in range(generations):
        nouvelle_population = []

        # Élites (10%)
        nb_elites = max(1, taille_pop // 10)
        elites = sorted(population, key=lambda ind: ind["cout"])[:nb_elites]
        nouvelle_population.extend(elites)

        # Création enfants
        while len(nouvelle_population) < taille_pop:
            p1 = selection(population)
            p2 = selection(population)
            enfant_ind = enfant(p1, p2)
            nouvelle_population.append(enfant_ind)

        population = nouvelle_population
        meilleur_gen = min(population, key=lambda ind: ind["cout"])

        if meilleur_gen["cout"] < meilleur["cout"]:
            meilleur = meilleur_gen

        #print(f"Génération {gen+1} | Meilleur coût = {meilleur['cout']:.4f}")

    return meilleur


In [53]:
resultat_ga = genetic_algorithm()

print("\n=== Résultat GA ===")
print("Coût :", resultat_ga["cout"])
print("Réplicas :", resultat_ga["r"])
print("VMs :", resultat_ga["x"])

y = placement_glouton(resultat_ga["r"], resultat_ga["x"])
print("Placement y_ij :\n", y)



=== Résultat GA ===
Coût : 13.1176
Réplicas : [41 83 58 46]
VMs : [ 0  0  0  0  1  0  0  8  0  0 10  0  0  0  0  0  0]
Placement y_ij :
 [[ 0  0  0  0  4  0  0 37  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 27  0  0 56  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 58  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 46  0  0  0  0  0  0]]


In [49]:
def simulated_annealing(
        T_init=50,
        T_min=0.1,
        alpha=0.95,
        iterations=60
    ):
    
    r, x = solution_initiale()
    best_cost = cout_total(x)
    best_r, best_x = r.copy(), x.copy()
    
    current_r, current_x = r.copy(), x.copy()
    current_cost = best_cost
    
    T = T_init

    while T > T_min:
        for _ in range(iterations):

            # --- Double Voisinage ---
            if random.random() < 0.85:
                r2, x2 = voisin(current_r, current_x)   # petit changement
            else:
                x2 = current_x.copy()                   # gros saut
                j = random.randint(0, NB_VM-1)
                x2[j] = random.randint(0, DISPO_VM[j])
                r2 = current_r.copy()

            # réparation
            r2, x2 = reparer(r2, x2)
            if r2 is None:
                continue

            new_cost = cout_total(x2)
            delta = new_cost - current_cost

            # acceptation
            if delta < 0 or random.random() < np.exp(-delta/T):
                current_r, current_x = r2, x2
                current_cost = new_cost

                if new_cost < best_cost:
                    best_cost = new_cost
                    best_r, best_x = r2.copy(), x2.copy()

        T *= alpha

    y = placement_glouton(best_r, best_x)
    return best_r, best_x, y, best_cost


In [54]:
applications = [1, 2, 5, 10]
resultats = {}

for k in applications:
    print(f"\n=== Optimisation pour {k} applications ===")

    # Mettre à jour lambdas
    lamb_k = k * lamb

    # Mettre à jour la fonction de réplicas minimaux
    def replicas_minimales():
        return np.ceil(lamb_k / mu).astype(int)

    # ============================
    # HILL CLIMBING
    # ============================
    try:
        r_hc, x_hc, y_hc, cost_hc = hill_climbing()
    except Exception as e:
        print(f"[HC] Erreur pour {k} applications :", e)
        r_hc = x_hc = y_hc = cost_hc = None

    # ============================
    # GENETIC ALGORITHM
    # ============================
    try:
        res_ga = genetic_algorithm()
        r_ga = res_ga["r"]
        x_ga = res_ga["x"]
        cost_ga = res_ga["cout"]
    except Exception as e:
        print(f"[GA] Erreur pour {k} applications :", e)
        r_ga = x_ga = cost_ga = None

    # ============================
    # SIMULATED ANNEALING
    # ============================
    try:
        r_sa, x_sa, y_sa, cost_sa = simulated_annealing()
    except Exception as e:
        print(f"[SA] Erreur pour {k} applications :", e)
        r_sa = x_sa = y_sa = cost_sa = None

    # Sauvegarde des résultats
    resultats[k] = {
        "HC_cost": cost_hc,
        "HC_r": r_hc,
        "HC_x": x_hc,
        
        "GA_cost": cost_ga,
        "GA_r": r_ga,
        "GA_x": x_ga,

        "SA_cost": cost_sa,
        "SA_r": r_sa,
        "SA_x": x_sa
    }

    # Affichage des résultats pour ce k
    print("  - HC cost :", cost_hc)
    print("  - GA cost :", cost_ga)
    print("  - SA cost :", cost_sa)



=== Optimisation pour 1 applications ===
  - HC cost : 1.7648
  - GA cost : 1.5721999999999998
  - SA cost : 1.7648

=== Optimisation pour 2 applications ===
  - HC cost : 2.6471999999999998
  - GA cost : 2.6471999999999998
  - SA cost : 2.6471999999999998

=== Optimisation pour 5 applications ===
  - HC cost : 7.0592
  - GA cost : 6.4832
  - SA cost : 7.0592

=== Optimisation pour 10 applications ===
  - HC cost : 14.5064
  - GA cost : 13.1176
  - SA cost : 15.2376


In [39]:
def simulated_annealing(
        T_init=10.0,         # température initiale
        T_min=0.001,         # température minimale
        alpha=0.97,          # taux de refroidissement
        iterations=300       # itérations par température
    ):
    
    # -------- Solution initiale --------
    r, x = solution_initiale()
    y = placement_glouton(r, x)
    best_r, best_x, best_y = r.copy(), x.copy(), y.copy()
    best_cost = cout_total(x)
    
    current_r, current_x, current_y = r.copy(), x.copy(), y.copy()
    current_cost = best_cost
    
    T = T_init
    
    # -------- Boucle du recuit simulé --------
    while T > T_min:
        for _ in range(iterations):
            
            # Générer voisin
            r2, x2 = voisin(current_r, current_x)
            
            # Réparer solution
            r2, x2 = reparer(r2, x2)
            if r2 is None:
                continue
            
            y2 = placement_glouton(r2, x2)
            if y2 is None:
                continue
            
            new_cost = cout_total(x2)
            delta = new_cost - current_cost
            
            # -------- Critère d'acceptation --------
            if delta < 0:
                # Meilleure solution → accepter
                current_r, current_x, current_y = r2, x2, y2
                current_cost = new_cost
                
                # Update best
                if new_cost < best_cost:
                    best_r, best_x, best_y = r2, x2, y2
                    best_cost = new_cost
            
            else:
                # Accepter une mauvaise solution avec proba exp(-delta / T)
                prob = np.exp(-delta / T)
                if random.random() < prob:
                    current_r, current_x, current_y = r2, x2, y2
                    current_cost = new_cost
        
        # Refroidissement
        T *= alpha
    
    return best_r, best_x, best_y, best_cost


In [58]:
r_sa, x_sa, y_sa, cost_sa = simulated_annealing()

print("=== Résultat Simulated Annealing ===")
print("Coût :", cost_sa)
print("Réplicas :", r_sa)
print("VMs :", x_sa)
print("Placement y_ij :\n", y_sa)


=== Résultat Simulated Annealing ===
Coût : 18.1296
Réplicas : [41 84 58 46]
VMs : [ 0  1  0  0  7  0  0  0  1  1 10  4  0  0  0  0  0]
Placement y_ij :
 [[ 0  2  0  0 28  0  0  0  8  3  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  5 79  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 58  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 23 23  0  0  0  0  0]]
